# Libs

In [10]:
import math
import warnings

import numpy as np
import arviz as az
import pymc3 as pm
import pandas as pd
import seaborn as sns
import scipy.special as sc
import theano.tensor as tt
import matplotlib.pyplot as plt
from scipy import stats
#from theano import shared, function
#from pymc3.distributions.dist_math import betaln, bound

from custom import pert

warnings.simplefilter(action="ignore", category=FutureWarning)

In [11]:
%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")
print(f"Running on PyMC v{pm.__version__}")
print(f"Running on ArviZ v{az.__version__}")

Running on PyMC v3.11.4
Running on ArviZ v0.11.2


# Main

## Baskets

In [12]:
with pm.Model():
    bs_pp = pert(name="Basket Size: Pure Players", a=1, b=2, c=5, transform=None).random(size=100000) # basket size distribution for pure players
    bs_bc = pert(name="Basket Size: Bricks & Clicks", a=5, b=45, c=100, transform=None).random(size=100000) # basket size distribution for bricks & clicks

## Passenger Car

In [13]:
car_milage = 2400000 # avg passenger car milage (km)
car_weight = 1000 # avg passenger car weight in (kg)

car_maint = 1080 # GHG per car caused by maintenance (kg)
car_maint_ghg = car_maint / car_milage # GHG per every km caused by maintenance (kg/km)

car_emis = 7 # GHG per kg of car weight caused by direct emissions (kg/kg)
car_emis_ghg = car_emis * car_weight / car_milage # GHG per every km caused by direct emissions (kg/km)

car_total_ghg = car_maint_ghg + car_emis_ghg

## Pure Players

### Last Mile Packaging

In [16]:
with pm.Model():
    product_vol = pm.Lognormal(name = 'Product Weight', mu=-1.38, sigma=1.14).random(size=100000) # volume of product in liters (l)
    box_sat = pert(name='Box Saturation', a=.1, b=.5, c=.9, transform=None).random(size=100000) # share of box volume consumed by product (l/l)

In [ ]:
cardboard_weight = 12.5 # grams of cardboard per every liter of box volume (g/l)
lm_box = cardboard_weight * product_vol / box_sat # distribution of weights of cardbord per single package (g)

